### Sample program for Random Forest Classifier  

#### Import libraries  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from joblib import dump

#### Parameters  

In [ ]:
csv_in = 'titanic.csv'

#### Read CSV file  

PassengerID: 乗客ID  
Survived: (1:survived, 2:dead)  生存結果 (1: 生存, 2: 死亡)  
Pclass: class (grade) of passenger (1: highest)  乗客の階級 1が最高級  
Name: 乗客の名前  
Sex: 性別  
Age: 年齢  
SibSp: #brothers/sisters + #spouse 一緒に乗船していた兄弟＋配偶者の数  
Parch: #parents + #children 一緒に乗船していた両親＋子供の数  
Ticket: チケット番号  
Fare: 乗船料金  
Cabin:  Cabin ID  部屋番号  
Embarked: (Cherbourg、Queenstown、Southampton) name of embarked port 乗船した港名  

In [ ]:
df = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
print(df.shape)
print(df.info())
display(df.head())

#### Drop rows with missing value at Age and Embarked columns  

In [ ]:
df = df.dropna(subset=['Age','Embarked']).reset_index(drop=True)

#### Extract data  

In [ ]:
df_X= df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
df_X = pd.get_dummies(df_X, drop_first=True)
ser_y = df['Survived']

In [ ]:
print(df_X.shape)
print(df_X.info())
display(df_X.head())
print(ser_y.shape)
print(ser_y.head())

#### Grid search (n_estimators, max_Depth of RF)  

In [ ]:
rfc=RandomForestClassifier(random_state=0)            

# Hyper paramet settings for grid search
param_grid = {
    'n_estimators': [50, 100, 500, 1000],
    'max_depth' : [2, 4, 6, None],
}

In [ ]:
grid_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=7)
gen_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)

In [ ]:
gs = GridSearchCV(rfc, param_grid , cv=grid_cv, scoring='accuracy')

In [ ]:
%%time
nested_score = cross_val_score(gs, X=df_X, y=ser_y, cv=gen_cv,
                               scoring='accuracy')
print(nested_score)
print(nested_score.mean())

**Expected is about 77.5% on average for unknown data (generalization performance)**
**未知データに対して期待される予測精度(汎化性能)は 平均 77.5% 程度**  

#### Cross-validation to obtain the model with the best hyperparameter set (best estimator)  
- Note: gs_best is already fit to the whole data (df_X) in gs.fit(df_X,ser_y)  

In [ ]:
%%time
gs.fit(df_X, ser_y)
gs_best = gs.best_estimator_

In [ ]:
print(gs_best)

**Best hyperparameters: n_estimators:500 , max_depth:4**  

#### Show feature importances of the best model  

In [ ]:
print(pd.Series(gs_best.feature_importances_, index=df_X.columns))

In [ ]:
plt.bar(df_X.columns, gs_best.feature_importances_)
plt.ylabel('Importance')
plt.xticks(rotation=90)
plt.show()

#### Save the best predictor  

In [ ]:
tag = 'titanic'
model_file = 'rfc_best_{}.joblib'.format(tag)
dump(gs_best, model_file)